<a href="https://colab.research.google.com/github/chelynl/Machine_Learning/blob/main/16_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Networks (ANN)
- Can fall under all 3 categories (supervised, unsupervised, reinforcement learning)
- **Supervised**
 - CNNs classifying images in imagenet
- **Unsupervised**
 - Boltzmann Machines, AutoEncoders, GANs, DC-GANS, VAE, SOMs, etc.
- **Reinforcement learning**
 - Deep Convolutional Q-Learning that plays videogames from pixel input, AlphaGO, etc. (deep reinforcement learning, "AI")

In [224]:
# load dependencies
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf

In [2]:
# see tf version
tf.__version__

'2.5.0'

## Data Preprocessing

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/chelynl/ML_notes/main/deep_learning/Churn_Modelling.csv?token=AMGO4MC3JY4W7WHLIVTAEQLA5R6CC')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
# See dimensions
df.shape

(10000, 14)

In [20]:
# No missing data 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [234]:
# Define target and inputs
X = df.iloc[:, 3:-1]
y = df.iloc[:, -1]

In [58]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [34]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

**Encode categorical data**

- Get binary values for the `'Gender'` column by using `OrdinalEncoder()` since `LabelEncoder()` is not compatible for features and it's usually used for binary targets.
- Get dummy values for the `'Geography'` column using `OneHotEncoder()` since there are three countries (levels) with no ordering.

After encoding, scale features because NNs perform better that way.

In [235]:
# Create encoding pipeline
encoder = make_column_transformer(
    # label encode "gender" col
    (OrdinalEncoder(), ['Gender']),
    # one-hot encode "geography" col
    (OneHotEncoder(), ['Geography']),
    remainder='passthrough'
)

# Encode features
X = encoder.fit_transform(X)
# Scale features
ss = StandardScaler()
X = ss.fit_transform(X)

# See number of units to use for NN
input_shape = [X.shape[1]]
print("Input shape: {}".format(input_shape))

# Partition data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Input shape: [12]


## Model Building

In [237]:
# Instantiate ANN
ann = tf.keras.models.Sequential()

# Add input layer and first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Add second input layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Add output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Model Training

In [238]:
# Compile the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fit ANN to training data
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.7586 - accuracy: 0.4921
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5995 - accuracy: 0.7952
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4965 - accuracy: 0.7962
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4496 - accuracy: 0.8027
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4275 - accuracy: 0.8124
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4130 - accuracy: 0.8227
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4011 - accuracy: 0.8303
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3899 - accuracy: 0.8366
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3798 - accuracy: 0.8406
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3712 - accura

## Make predictions and evaluate model

Use our ANN model to predict if the customer with the following informations will leave the bank:

- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: \$60000
- Number of Products: 2
- Does this customer have a credit card ? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: \$50000

So, should we say goodbye to that customer ?

In [239]:
# Get prediction

"""
Feature values should be in a double square brackets because the "predict" 
method always expects a 2D array as the format of its inputs. 

Also, one-hot encoding was done manually prior to scaling.
"""

print(ann.predict(ss.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


The ANN model predicts that this customer stays in the bank!



In [240]:
# Get test predictions
y_pred = ann.predict(X_test) # class probabilities
y_pred = (y_pred > 0.5) # class labels

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.values.reshape(len(y_test),1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [241]:
# See confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1514   81]
 [ 198  207]]


0.8605